Initial dip model

Issue: Mathematical erorr. Factored out $\epsilon(a, s)$ from the integral $\int_0^a{(1+\epsilon(a, s))\mu(s)ds}$ without integrating.

In [ ]:
from hill import *
import numpy as np
from scipy.integrate import quad
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from firm_dynamics import obtain_survival_fractions, obtain_total_alive_count

def epsilon(a, eps0, tau, t_e):
    return eps0 * np.exp(-(a - t_e) / tau)

def hill_survival_with_dip(a, mu_ub, mu_lb, K, m, t_e, delta, eps0, tau):

    if a < t_e - delta:
        return hill_survival_function(a, mu_ub, mu_lb, K, m)
    
    elif t_e - delta <= a <= t_e:
        val, _ = quad(lambda s: hill_hazard(s, mu_ub, mu_lb, K, m), 0, delta-(t_e-a))
        return hill_survival_function(a, mu_ub, mu_lb, K, m) * np.exp(-epsilon(a, eps0, tau, t_e) * val)
    
    else:
        val, _ = quad(lambda s: hill_hazard(s, mu_ub, mu_lb, K, m), a-t_e, a-t_e+delta)
        return hill_survival_function(a, mu_ub, mu_lb, K, m) * np.exp(-epsilon(a, eps0, tau, t_e) * val)

def model_hill_with_dip(ages, mu_ub, mu_lb, K, m, t_e, delta, eps0, tau):
    return np.array([hill_survival_with_dip(a, mu_ub, mu_lb, K, m, t_e, delta, eps0, tau) for a in ages])

def neg_ll_hill_with_dip(params, ages, survivors, totals):
    mu_ub, mu_lb, K, m, t_e, delta, eps0, tau = params
    ll = 0
    if mu_lb < 0 or mu_ub < mu_lb or K <= 0 or m <= 0 or t_e < 0 or delta < 0:
        return np.inf

    S_vals = model_hill_with_dip(ages, mu_ub, mu_lb, K, m, t_e, delta, eps0, tau)
    S_vals = np.clip(S_vals, 1e-12, 1 - 1e-12)  # avoid log(0)

    deaths = totals - survivors
    logL = np.sum(survivors * np.log(S_vals) + deaths * np.log(1 - S_vals))
    return -logL  # minimize negative log-likelihood

def fit_hill_with_dip(ages, survivors, totals, initial_guess = [0.1, 0.1, 7, 10, 8, 2.5, 3, 0.8]):
    from scipy.optimize import minimize

    bounds = [
        (0.01, 0.3),   # mu_ub
        (1e-6, 0.3),   # mu_lb
        (0.1, 30),     # K
        (0.5, 100),    # m
        (3, 10),       # t_e
        (0.1, 8),      # delta
        (0.1, 30),    # eps0
        (0.01, 30)      # tau
    ]
    result = minimize(neg_ll_hill_with_dip, initial_guess, args=(ages, survivors, totals), bounds=bounds)

    return result

def lsq_hill_with_dip(params, ages, survival_fractions):
    mu_ub, mu_lb, K, m, t_e, delta, eps0, tau = params
    model = model_hill_with_dip(ages, mu_ub, mu_lb, K, m, t_e, delta, eps0, tau)
    model = np.clip(model, 1e-12, 1 - 1e-12)
    return np.sum((survival_fractions - model) ** 2)

sector_list = ['G', 'M', 'F', 'J', 'K', 'C', 'H', 'S', 'N', 'I', 'P', 'L', 'Q', 'R']
parameters = [
    [0.13660027, 0.03574423, 12.39113424, 4.14356328],
    [0.10877533, 0.0418096, 12.55269306, 4.41391222],
    [0.079990154, 1.00E-10, 26.18237719, 79.99986416],
    [0.13090805, 0.03791174, 11.9429807, 4.05657508],
    [0.070120134, 0.011071032, 17.60063205, 11.71975389],
    [0.10301031, 0.04128293, 9.26045477, 8.13925264],
    [0.190143914, 0.028016019, 6.93767599, 100],
    [0.14058029, 1.00E-10, 12.9535533, 5.1898739],
    [0.12396223, 1.00E-10, 16.4327672, 3.67640026],
    [0.12568692, 0.03447114, 17.44283135, 5.60609428],
    [0.121213526, 0.068684245, 9.44518567, 100],
    [0.074121126, 1.00E-10, 25.77531849, 79.99860496],
    [0.078301599, 0.047197935, 7.79197632, 100],
    [0.132289514, 0.085485775, 8.85732298, 100]
]
sector_params_MLE = dict(zip(sector_list, parameters))

for sector in sector_list:
    survival_fractions, ages = obtain_survival_fractions(df_analysis, 'Sector', sector)
    totals, survivors = obtain_total_alive_count(df_analysis, 'Sector', sector)

    initial_guess1, initial_guess2 = [0.13, 0.05, 9, 10, 8, 0.3, 10, 0.4], [0.12, 0.06, 7, 10, 8, 2.5, 10.0, 1.0]
    bounds = [
        (0.01, 0.3),   # mu_ub
        (1e-6, 0.3),   # mu_lb
        (0.1, 30),     # K
        (0.5, 100),    # m
        (3, 10),       # t_e
        (0.1, 8),      # delta
        (0.1, 20),    # eps0
        (0.01, 30)      # tau
    ]

    result1 = minimize(lsq_hill_with_dip, initial_guess1, args=(ages, survival_fractions), bounds=bounds)
    ll_1 = -neg_ll_hill_with_dip(result1.x, ages, survivors, totals)
    result2 = minimize(lsq_hill_with_dip, initial_guess2, args=(ages, survival_fractions), bounds=bounds)
    ll_2 = -neg_ll_hill_with_dip(result2.x, ages, survivors, totals)

    # result1 = fit_hill_with_dip(ages, survivors, totals, initial_guess1)
    # ll_1 = -neg_ll_hill_with_dip(result1.x, ages, survivors, totals)
    # result2 = fit_hill_with_dip(ages, survivors, totals, initial_guess2)
    # ll_2 = -neg_ll_hill_with_dip(result2.x, ages, survivors, totals)


    result = result1 if ll_1 > ll_2 else result2
    mu_ub, mu_lb, K, m, t_e, delta, eps0, tau = result.x

    aic_with_dip = 2 * 8 + 2 * neg_ll_hill_with_dip(result.x, ages, survivors, totals)
    aic_original = 2 * 4 + 2 * neg_log_likelihood_hill(sector_params_MLE[sector], ages, survivors, totals)
    print(f'AIC for sector {sector}')
    print(f'with dip: {aic_with_dip}, original: {aic_original}')
    print(f'Parameters for sector {sector}')
    print(f'mu_ub: {mu_ub}, mu_lb: {mu_lb}, K: {K}, m: {m}, t_e: {t_e}, delta: {delta}, eps0: {eps0}, tau: {tau}')

    plt.plot(ages, survival_fractions, 'o', markersize=3)
    plt.plot(ages, model_hill_with_dip(ages, mu_ub, mu_lb, K, m, t_e, delta, eps0, tau), label='with dip')
    plt.plot(ages, model_survival_curve_hill(ages, *sector_params_MLE[sector]), '--', label = 'original MLE fit')
    # plt.plot(ages, model_survival_curve_hill(ages, mu_ub, mu_lb, K, m), label='without dip')
    plt.title(f'Sector {sector} fit with dip')
    plt.legend()
    plt.show()

Idea was do pairwise comparison of death models between regions, to see if they could be grouped.  

Issue: Wrong calculation of log likelihood values, it assumed that death rate was constant across firm age.

In [ ]:
def plot_age_distribution(df, *, year='2023', sector=None, label=None):
    """
    Plot the age distribution of firms.
    
    Parameters:
    df: DataFrame containing the firms.
    year: The year to filter the firms by.
    sector: The sector to filter the firms by (optional).
    label: Label for the plot (optional).
    """
    import matplotlib.pyplot as plt
    import numpy as np
    
    # Ensure 'Entry Date' and 'Exit Date' are datetime objects
    df['Entry Date'] = pd.to_datetime(df['Entry Date'])
    df['Exit Date'] = pd.to_datetime(df['Exit Date'], errors='coerce')
    # Convert year to datetime object
    date = pd.to_datetime(year + '-01-01') 

    # Filter firms alive at the given date
    df_filtered = df.copy()
    df_filtered = df[
        (df['Entry Date'] < date) &
        ((df['Exit Date'].isna()) | (df['Exit Date'] > date))
    ].copy()
    df_filtered['Exit Date'] = date
    df_filtered['Age'] = (date - df_filtered['Entry Date']).dt.days / 365.25

    # Filter by sector if provided
    if sector is not None:
        df_filtered = df_filtered[df_filtered['Sector'] == sector]
    
    # Line plot for age density
    counts, bin_edges = np.histogram(df_filtered['Age'].dropna(), bins=50)
    # Calculate bin midpoints
    bin_midpoints = (bin_edges[:-1] + bin_edges[1:]) / 2
    plt.plot(
        bin_midpoints,
        counts / counts.sum(),  # Normalize to get density
        label=label
    )

def log_likelihood(mu, alive_counts, dead_counts):
    phi = 1 - mu # Say phi is the survival probability (i.e. phi = 1 - mu)
    log_p = 0
    for i in range(1, len(alive_counts)):
        log_p += dead_counts[i]*(np.log(1-pow(phi, i))) + alive_counts[i] * i * np.log(phi) + math.log(math.comb(alive_counts[i] + dead_counts[i], dead_counts[i]))
    return log_p

def neg_log_likelihood(params, alive_counts1, dead_counts1, alive_counts2, dead_counts2):
    mu1, mu2 = params
    ll1 = log_likelihood(mu1, alive_counts1, dead_counts1)
    ll2 = log_likelihood(mu2, alive_counts2, dead_counts2)
    return -(ll1 + ll2)  # negative for minimization

# Function to perform the likelihood ratio test
# 
def likelihood_ratio_test(df_a, df_b):
    """
    Perform the likelihood ratio test to compare the death rates across two dataframes.
    where H0: Death rate is constant across df_a and df_b
    and H1: Death rate is not constant across df_a and df_b
    
    Parameters:
    df_a: DataFrame containing the first set of firms.
    df_b: DataFrame containing the second set of firms.
    
    Returns:
    lrt_statistic: Likelihood ratio test statistic
    p_value: p-value for the test
    """
    
    df1 = df_a.copy()
    df2 = df_b.copy()

    # Create list of 31 years from 1993 to 2023
    years = np.arange(1993, 2024, 1)
    years = years[::-1]

    # Convert Entry Date to the nearest year
    df1['Entry Date'] = df1['Entry Date'].dt.year + (df1['Entry Date'].dt.month >= 7).astype(int)
    df2['Entry Date'] = df2['Entry Date'].dt.year + (df2['Entry Date'].dt.month >= 7).astype(int)

    # Create list of counts of alive/dead firms for each year for NR
    alive_counts_1 = []
    dead_counts_1 = []
    alive_counts_2 = []
    dead_counts_2 = []
    for year in years:
        alive_count1 = df1[
            (df1['Entry Date'] == year) &
            (df1['Exit Date'].isna())
        ].shape[0]
        alive_counts_1.append(alive_count1)

        dead_count1 = df1[
            (df1['Entry Date'] == year) &
            (df1['status'] == 0)
        ].shape[0]
        dead_counts_1.append(dead_count1)

        alive_count2 = df2[
            (df2['Entry Date'] == year) &
            (df2['Exit Date'].isna())
        ].shape[0]
        alive_counts_2.append(alive_count2)

        dead_count2 = df2[
            (df2['Entry Date'] == year) &
            (df2['status'] == 0)
        ].shape[0]
        dead_counts_2.append(dead_count2)

    # Calculate the MLE for H0
    mu_mle = minimize_scalar(
        lambda mu: -log_likelihood(mu, alive_counts_1, dead_counts_1) - log_likelihood(mu, alive_counts_2, dead_counts_2),
        bounds=(0.01, 0.99),
        method='bounded'
    ).x

    # Calculate the log-likelihood for H0
    ll_h0 = log_likelihood(mu_mle, alive_counts_1, dead_counts_1) + log_likelihood(mu_mle, alive_counts_2, dead_counts_2)

    # Calculate the MLE for H1
    result = minimize(
        neg_log_likelihood,
        x0=[0.1, 0.1],  # initial guess for mu1 and mu2
        args=(alive_counts_1, dead_counts_1, alive_counts_2, dead_counts_2),
        bounds=[(0.01, 0.99), (0.01, 0.99)]
    )
    mu_1_mle, mu_2_mle = result.x

    # Calculate the log-likelihood for H1
    ll_h1 = log_likelihood(mu_1_mle, alive_counts_1, dead_counts_1) + log_likelihood(mu_2_mle, alive_counts_2, dead_counts_2)

    # Calculate the likelihood ratio test statistic
    lrt_statistic = 2 * (ll_h1 - ll_h0)

    # Calculate the p-value for the likelihood ratio test
    p_value = chi2.sf(lrt_statistic, df=1)

    return lrt_statistic, p_value, mu_mle, mu_1_mle, mu_2_mle

# Create filtered dataframes
def filter_dataframe(category, filter, *, category2 = None, filter2 = None):
    """
    Filter the df_analysis dataframe based on the given category and filter.
    If category2 and filter2 are provided, it will filter based on both conditions.
    """
    if category2 is not None and filter2 is not None:
        df = df_analysis.copy()
        df = df[(df[category] == filter) & (df[category2] == filter2)]
    else:
        df = df_analysis.copy()
        df = df[df[category] == filter]
        
    return df

def plot_survival_fractions(df, label):
    """
    Plot survival fractions against age.
    """

    # Create list of ages from 0 to 60
    ages_sf = np.arange(0, 61, 1)
    # Create list of 31 years from 1963 to 2023
    years = np.arange(1963, 2024, 1)
    years = years[::-1]

    # Convert Entry Date to nearest year
    df['Entry Date'] = df['Entry Date'].dt.year + (df['Entry Date'].dt.month >= 7).astype(int)

    # Create list of alive/dead firms for each year
    alive_counts = []
    dead_counts = []
    for year in years:
        alive_count = df[
            (df['Entry Date'] == year) &
            (df['Exit Date'].isna())
        ].shape[0]
        alive_counts.append(alive_count)

        dead_count = df[
            (df['Entry Date'] == year) &
            (df['status'] == 0)
        ].shape[0]
        dead_counts.append(dead_count)

    # Create list of survival fractions
    survival_fractions = np.array(alive_counts) / (np.array(alive_counts) + np.array(dead_counts))

    plt.plot(ages_sf, survival_fractions, label=label, marker='o', markersize=3)

def plot_log_survival_fractions(df, label):
    """
    Plot survival fractions against age.
    """

    # Create list of ages from 0 to 30
    ages_sf = np.arange(0, 31, 1)
    # Create list of 31 years from 1993 to 2023
    years = np.arange(1993, 2024, 1)
    years = years[::-1]

    # Convert Entry Date to nearest year
    df['Entry Date'] = df['Entry Date'].dt.year + (df['Entry Date'].dt.month >= 7).astype(int)

    # Create list of alive/dead firms for each year
    alive_counts = []
    dead_counts = []
    for year in years:
        alive_count = df[
            (df['Entry Date'] == year) &
            (df['Exit Date'].isna())
        ].shape[0]
        alive_counts.append(alive_count)

        dead_count = df[
            (df['Entry Date'] == year) &
            (df['status'] == 0)
        ].shape[0]
        dead_counts.append(dead_count)

    # Create list of survival fractions
    survival_fractions = np.array(alive_counts) / (np.array(alive_counts) + np.array(dead_counts))

    plt.plot(ages_sf, np.log(survival_fractions), label=label, marker='o', markersize=3)

    log_survival = np.log(survival_fractions)

    # Fit linear model (least squares)
    coeffs = np.polyfit(ages_sf, log_survival, deg=1)
    predicted = np.polyval(coeffs, ages_sf)
    plt.plot(ages_sf, predicted, label=f'Linear Fit ({label})', linestyle='--')

    # Compute R^2
    ss_res = np.sum((log_survival - predicted)**2)
    ss_tot = np.sum((log_survival - np.mean(log_survival))**2)

    r_squared = 1 - (ss_res / ss_tot)
    print(f"R²: {r_squared}")


def plot_exponential(mu, *, label='Exponential Fit', linestyle='--'):
    """
    Plot the exponential survival function S(age) = exp(-mu * age).
    """
    ages = np.arange(0, 61, 1)
    survival_fractions = np.exp(-mu * ages)
    plt.plot(ages, survival_fractions, label=label, linestyle=linestyle)

def log_likelihood_power(params, alive_counts, dead_counts):
    log_p = 0
    a, b = params
    for i in range(1, len(alive_counts)):
        phi = 1 - a/(1+b*i) # Say phi is the survival probability (i.e. phi = 1 - mu)
        log_p += dead_counts[i]*(np.log(1-pow(phi, i))) + alive_counts[i] * i * np.log(phi) + math.log(math.comb(alive_counts[i] + dead_counts[i], dead_counts[i]))
    return log_p


"""
Power law model:
"""

# Function to perform the likelihood ratio test
def likelihood_ratio_test_power(df_a, df_b, *, initial=[0.1,0.01,0.1,0.01]):
    """
    Perform the likelihood ratio test to compare the death rates across two dataframes.
    where H0: Death rate is constant across df_a and df_b
    and H1: Death rate is not constant across df_a and df_b
    
    Parameters:
    df_a: DataFrame containing the first set of firms.
    df_b: DataFrame containing the second set of firms.
    
    Returns:
    lrt_statistic: Likelihood ratio test statistic
    p_value: p-value for the test
    """
    
    df1 = df_a.copy()
    df2 = df_b.copy()

    # Create list of 31 years from 1993 to 2023
    years = np.arange(1993, 2024, 1)
    years = years[::-1]

    # Convert Entry Date to the nearest year
    df1['Entry Date'] = df1['Entry Date'].dt.year + (df1['Entry Date'].dt.month >= 7).astype(int)
    df2['Entry Date'] = df2['Entry Date'].dt.year + (df2['Entry Date'].dt.month >= 7).astype(int)

    # Create list of counts of alive/dead firms for each year for NR
    alive_counts_1 = []
    dead_counts_1 = []
    alive_counts_2 = []
    dead_counts_2 = []
    for year in years:
        alive_count1 = df1[
            (df1['Entry Date'] == year) &
            (df1['Exit Date'].isna())
        ].shape[0]
        alive_counts_1.append(alive_count1)

        dead_count1 = df1[
            (df1['Entry Date'] == year) &
            (df1['status'] == 0)
        ].shape[0]
        dead_counts_1.append(dead_count1)

        alive_count2 = df2[
            (df2['Entry Date'] == year) &
            (df2['Exit Date'].isna())
        ].shape[0]
        alive_counts_2.append(alive_count2)

        dead_count2 = df2[
            (df2['Entry Date'] == year) &
            (df2['status'] == 0)
        ].shape[0]
        dead_counts_2.append(dead_count2)

    # H0: a, b are the same for both groups
    def neg_ll_h0(params):
        return -(log_likelihood_power(params, alive_counts_1, dead_counts_1) +
                 log_likelihood_power(params, alive_counts_2, dead_counts_2))
    result_h0 = minimize(neg_ll_h0, x0=[0.1,0.01], bounds=[(1e-4, 1), (1e-4, 10)])
    a_mle, b_mle = result_h0.x
    ll_h0 = -neg_ll_h0([a_mle, b_mle])

   # H1: a, b are different for each group
    def neg_log_likelihood(params):
        a1, b1, a2, b2 = params
        ll1 = log_likelihood_power([a1, b1], alive_counts_1, dead_counts_1)
        ll2 = log_likelihood_power([a2, b2], alive_counts_2, dead_counts_2)
        return -(ll1 + ll2)  # negative for minimization
    result_h1 = minimize(
        neg_log_likelihood,
        x0=initial,
        bounds=[(1e-4, 1), (1e-4, 10), (1e-4, 1), (1e-4, 10)]
    )
    a1_mle, b1_mle, a2_mle, b2_mle = result_h1.x
    ll_h1 = -neg_log_likelihood([a1_mle, b1_mle, a2_mle, b2_mle])

    # Calculate the likelihood ratio test statistic
    lrt_statistic = 2 * (ll_h1 - ll_h0)

    # Calculate the p-value for the likelihood ratio test
    p_value = chi2.sf(lrt_statistic, df=2)

    return lrt_statistic, p_value, [a_mle, b_mle, a1_mle, b1_mle, a2_mle, b2_mle]


# Example usage
region_list = ['CR', 'NR', 'WR', 'ER', 'NER']
# for region1 in region_list:
region1 = 'CR'
df_a = filter_dataframe('Region', region1, category2='Sector', filter2='G')
df_b = filter_dataframe('Sector', 'G')

print(f'Sample size for df_a: {len(df_a)}')
print(f'Sample size for df_b: {len(df_b)}')

# Perform the likelihood ratio test
lrt_statistic, p_value, mu_mle, mu_1, mu_2 = likelihood_ratio_test(df_a, df_b)
print(f'Likelihood Ratio Test Statistic: {lrt_statistic}')
print(f'P-value for the likelihood ratio test: {p_value}')
print(f'MLE for mu: {mu_mle}') # Under H0
print(f'MLE for mu_a: {mu_1} and MLE for mu_b: {mu_2}') # Under H1

# Conclusion based on p-value
if p_value < 0.05:
    print("Reject H0: Death rates are different.")
else:
    print("Fail to reject H0: Death rates are not significantly different.")

# Plot survival fractions against age
plot_survival_fractions(df_a, label = region1)
plot_survival_fractions(df_b, label = 'Overall')
# Plot the exponential survival function
plot_exponential(mu_mle, label='Exponential Fit (H0)', linestyle='--')
plot_exponential(mu_1, label='Exponential Fit (H1, CR)')
plot_exponential(mu_2, label='Exponential Fit (H1, Overall)')

plt.title('Comparison of Survival Fractions')
plt.xlim(0, 30) # since survival fractions are approximately monotonic only to about age 30
plt.legend()
plt.show()

Regional model comparison

In [ ]:
region_list = ['NR','WR','CR','ER','NER']

for region in region_list:
    survival_frac, ages = obtain_survival_fractions(df_analysis, 'Region', region)
    result = fit_survival_model(ages, survival_frac, [0.2, 0.05, 8, 7])
    best_params = result.x
    predicted_curve = model_survival_curve_hill(ages, *best_params)
    print(f'{region}: {best_params}')

    # # Plot model against empirical
    # plt.figure(figsize=(8, 5))
    # plt.plot(ages, survival_frac, 'o', markersize = 3, label="Empirical")
    # plt.plot(ages, predicted_curve, '--', label="Fitted Model")
    # plt.plot(ages, overall_predicted_curve, '--', label="Overall Fitted Model")
    # plt.xlabel("Firm Age")
    # plt.ylabel("Survival Fraction")
    # plt.title(f"Model Fit: {region}")
    # plt.grid(True)
    # plt.legend()
    # plt.show()

In [ ]:
region_ll_hill = []
for fil in region_list:

    sf, ages = obtain_survival_fractions(df_analysis, 'Region', fil)

    totals, survivors = obtain_total_alive_count(df_analysis, 'Region', fil)

    mu_ub, mu_lb, K, m = fit_hill_model_mle(ages, survivors, totals).x
    
    print(f'{fil} {[float(x) for x in [mu_ub, mu_lb, K, m]]}')    

    # Plot fit
    fitted_S = model_survival_curve_hill(ages, mu_ub, mu_lb, K, m)

    # print(f'll for {fil}: {-neg_log_likelihood_hill([mu_ub, mu_lb, K, m], ages, survivors, totals)}')
    region_ll_hill.append(neg_log_likelihood_hill([mu_ub, mu_lb, K, m], ages, survivors, totals))

    plt.plot(ages, sf, 'o', markersize = 3, label='Observed survival fractions')
    plt.plot(ages, fitted_S, label='MLE Hill Model', linestyle='--')
    plt.title(f'MLE Hill fit for {fil}')
    plt.xlabel('Age (years)')
    plt.ylabel('Survival Fraction')
    plt.legend()
    plt.grid(True)
    plt.show()


In [ ]:
region_ll_power = []
for fil in region_list:

    sf, ages = obtain_survival_fractions(df_analysis, 'Region', fil)

    totals, survivors = obtain_total_alive_count(df_analysis, 'Region', fil)

    lam, alpha = fit_power_model_mle(ages, survivors, totals)

    # Plot fit
    fitted_S = model_survival_curve_power(ages, lam, alpha)

    print(f'll for {fil}: {-neg_log_likelihood_power([lam, alpha], ages, survivors, totals)}')
    region_ll_power.append(neg_log_likelihood_power([lam, alpha], ages, survivors, totals))

    print(f'params lam, alpha are {[float(x) for x in [lam, alpha]]}')

    plt.plot(ages, sf, 'o', markersize = 3, label='Observed survival fractions')
    plt.plot(ages, fitted_S, label='MLE power model', linestyle='--')
    plt.title(f'MLE power fit for {fil}')
    plt.xlabel('Age (years)')
    plt.ylabel('Survival Fraction')
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
region_ll_constant = []
region_list = ['NR','WR','CR','ER','NER']
for fil in region_list:
    sf, ages = obtain_survival_fractions(df_analysis, 'Region', fil)
    totals, survivors = obtain_total_alive_count(df_analysis, 'Region', fil)
    result = fit_constant_model_mle(ages, survivors, totals)
    # print(result.x)
    # plt.plot(ages, sf, 'o', markersize = 3, label='Observed survival fractions')
    # plt.plot(ages, model_survival_curve_constant(ages, result.x[0]), label='MLE Constant Model', linestyle='--')
    # plt.title(f'MLE Constant fit for {fil}')
    # plt.xlabel('Age (years)')
    # plt.ylabel('Survival Fraction')
    # plt.legend()
    # plt.grid(True)
    # plt.show()
    region_ll_constant.append(result.fun)

region_aic_constant, region_bic_constant = compute_aic_bic(np.array(region_ll_constant), 1, len(ages))

In [ ]:
region_aic_hill, region_bic_hill = compute_aic_bic(np.array(region_ll_hill), 4, len(ages))
region_aic_power, region_bic_power = compute_aic_bic(np.array(region_ll_power), 2, len(ages))

region_df_aic = pd.DataFrame({
    'Region': region_list,
    'AIC_Hill': region_aic_hill,
    'AIC_Power': region_aic_power,
    'AIC_Hill3k': region_aic_hill3k,
    'AIC_Constant': region_aic_constant
})
region_df_aic.head()

region_df_bic = pd.DataFrame({
    'Region': region_list,
    'BIC_Hill': region_bic_hill,
    'BIC_Power': region_bic_power,
    'BIC_Hill3k': region_bic_hill3k,
    'BIC_Constant': region_bic_constant
})
region_df_bic.head()

In [ ]:
for fil in region_list:

    sf, ages = obtain_survival_fractions(df_analysis, 'Region', fil)

    totals, survivors = obtain_total_alive_count(df_analysis, 'Region', fil)

    mu_ub, mu_lb, K, m = fit_hill_model_mle(ages, survivors, totals).x

    print(f'params mu_ub, mu_lb, K, m for {fil} are {[float(x) for x in [mu_ub, mu_lb, K, m]]}')

    plt.plot(ages, hill_hazard(ages, mu_ub, mu_lb, K, m), label = fil, linestyle='--')
plt.title(f'Hill Function')
plt.xlabel('Age (years)')
plt.legend()
plt.grid(True)
plt.show()

Hill3k

In [ ]:
# Hill function with number of params k=3, where hill coefficient is set constant at m=5

def hill3k_hazard(a, mu_ub, mu_lb, K):
    m = 10
    return mu_ub - (mu_ub - mu_lb) * (a**m) / (a**m + K**m + 1e-10)  # Add epsilon to avoid div by 0

# ---- Step 2: Define survival function S(a) using quad integration ----

def hill3k_survival_function(a, mu_ub, mu_lb, K):
    result, _ = quad(lambda s: hill3k_hazard(s, mu_ub, mu_lb, K), 0, a)
    return np.exp(-result)

# ---- Step 3: Vectorized function to compute model survival over array of ages ----

def model_survival_curve_hill3k(ages, mu_ub, mu_lb, K):
    return np.array([hill3k_survival_function(a, mu_ub, mu_lb, K) for a in ages])

# Negative log-likelihood for binomial MLE
def neg_log_likelihood_hill3k(params, ages, survivors, totals):
    mu_ub, mu_lb, K = params
    if mu_lb < 0 or mu_ub < mu_lb or K <= 0:
        return np.inf

    S_vals = model_survival_curve_hill3k(ages, mu_ub, mu_lb, K)
    S_vals = np.clip(S_vals, 1e-12, 1 - 1e-12)  # avoid log(0)

    deaths = totals - survivors
    logL = np.sum(survivors * np.log(S_vals) + deaths * np.log(1 - S_vals))
    return -logL  # minimize negative log-likelihood

# Fit function
def fit_hill3k_model_mle(ages, survivors, totals, initial_guess=[0.12, 0.04, 10.0]):
    '''
    ages = vector of ages
    survivors = number of survivors in age bin
    totals = total number of firms in age bin
    initial_guess = [mu_ub, mu_lb, K]

    returns best params according to MLE
    
    '''
    bounds = [(0.01, 0.3), (1e-10, 0.15), (0.01, 30)]
    constraints = {'type': 'ineq', 'fun': lambda x: x[0] - x[1]}  # mu_ub >= mu_lb
    result = minimize(neg_log_likelihood_hill3k, initial_guess,
                      args=(ages, survivors, totals),
                      bounds=bounds, constraints=constraints)
    return result

In [ ]:
print("----Hill3k model fits (region)----")
region_list = ['NR','WR','CR','ER','NER']
region_ll_hill3k = []
for fil in region_list:
    sf, ages = obtain_survival_fractions(df_analysis, 'Region', fil)
    totals, survivors = obtain_total_alive_count(df_analysis, 'Region', fil)
    result = fit_hill3k_model_mle(ages, survivors, totals)
    print(f'{fil}: {[float(x) for x in result.x]}')
    # plt.plot(ages, sf, 'o', markersize = 3, label='Observed survival fractions')
    # plt.plot(ages, model_survival_curve_hill3k(ages, *result.x), label='MLE Hill3k Model', linestyle='--')
    # plt.title(f'MLE Hill3k fit for {fil}')
    # plt.xlabel('Age (years)')
    # plt.ylabel('Survival Fraction')
    # plt.legend()
    # plt.grid(True)
    # plt.show()
    region_ll_hill3k.append(result.fun)

print("----Hill3k model fits (sector)----")
sector_list = ['G', 'M', 'F', 'J', 'K', 'C', 'H', 'S', 'N', 'I', 'P', 'L', 'Q', 'R']
sector_ll_hill3k = []
parameters_3k = []
for fil in sector_list:
    sf, ages = obtain_survival_fractions(df_analysis, 'Sector', fil)
    totals, survivors = obtain_total_alive_count(df_analysis, 'Sector', fil)
    # [7.23785797e-02, 6.43020350e-03, 1.74446268e+01]
    result1 = fit_hill3k_model_mle(ages, survivors, totals, [0.1, 0.05, 10])
    result2 = fit_hill3k_model_mle(ages, survivors, totals, [0.2, 0.03, 7])

    result = result1 if result1.fun < result2.fun else result2
    print(f'{fil}: {[float(x) for x in result.x]}')
    # plt.plot(ages, sf, 'o', markersize = 3, label='Observed survival fractions')
    # plt.plot(ages, model_survival_curve_hill3k(ages, *result.x), label='MLE Hill3k Model', linestyle='--')
    # plt.title(f'MLE Hill3k fit for {fil}')
    # plt.xlabel('Age (years)')
    # plt.ylabel('Survival Fraction')
    # plt.legend()
    # plt.grid(True)
    # plt.show()
    sector_ll_hill3k.append(result.fun)
    parameters_3k.append([float(x) for x in result.x])
print(parameters_3k)


region_aic_hill3k, region_bic_hill3k = compute_aic_bic(np.array(region_ll_hill3k), 3, len(ages))
sector_aic_hill3k, sector_bic_hill3k = compute_aic_bic(np.array(sector_ll_hill3k), 3, len(ages))